In [ ]:
from __future__ import print_function
import sys
import rospy
import time
import tf
import jupyros as jr
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
import second_assignment.msg
from second_assignment.msg import Pos_vel
from second_assignment.msg import PlanningAction, PlanningGoal, PlanningResult
from std_srvs.srv import SetBool
from actionlib_msgs.msg import GoalStatus
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tf.transformations import quaternion_matrix
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox, HBox   

In [ ]:
global x
global y 
global client
global var
global goal_reached
global latest_feedback
global goal_list
global num_reached_goals
global num_goal_not_reached
global first_goal

num_reached_goals = 0
num_goal_not_reached = 0
first_goal = False
goal_list = []

rospy.init_node('node_action_client')
client = actionlib.SimpleActionClient('/reaching_goal', second_assignment.msg.PlanningAction)

In [ ]:
def feedback_cb(feedback):
    global latest_feedback
    latest_feedback = feedback

In [ ]:
# def pub_pos_vel(message):
    
#     """
#     Callback function used to publish the position and velocity of the robot to the topic /pos_vel_topic using a custom message type Pos_vel
    
#     args:
#         message: message received from the topic /odom type Odometry
#     """
#     # Creates a publisher to the topic /pos_vel_topic which uses the custom message type Pos_vel
#     pub = rospy.Publisher('pos_vel_topic', Pos_vel, queue_size=1)

#     # Obtained by looking at the message type of the topic /odom (Odometry message) 
#     # that contains header 2 pose and 2 twist 

#     pos_vel = Pos_vel()
#     pos_vel.x = message.pose.pose.position.x
#     pos_vel.y = message.pose.pose.position.y
#     pos_vel.vel_x = message.twist.twist.linear.x
#     pos_vel.vel_z = message.twist.twist.angular.z

#     pub.publish(pos_vel) 

In [ ]:
pub = rospy.Publisher('goal_list', Pos_vel, queue_size=10)
x_input = widgets.Text(description='x coordinate:', continuous_update=False)
y_input = widgets.Text(description='y coordinate:', continuous_update=False)
button = widgets.Button(description="Confirm")

x, y = None, None
goal_list = []

def handle_x_input(change):
    global x
    x = float(change.new)

def handle_y_input(change):
    global y
    y = float(change.new)

def on_button_clicked(b):
    global x, y, first_goal
    print("User input for x:", x)
    print("User input for y:", y)
    first_goal = True
    send_goal()

x_input.observe(handle_x_input, names='value')
y_input.observe(handle_y_input, names='value')

button.on_click(on_button_clicked)

def send_goal():
    global x
    global y
    global goal_list

    client.wait_for_server()
    
    rospy.set_param('/des_pos_x', x)
    rospy.set_param('/des_pos_y', y)
    
    pos_vel = Pos_vel()
    pos_vel.x = x
    pos_vel.y = y

    goal_list.append([x,y])
    
    # Publish the goal to the topic goal_list to be displayed in the map
    pub.publish(pos_vel)

    goal = second_assignment.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    client.send_goal(goal, feedback_cb=feedback_cb)


cancel_button = widgets.Button(description="Cancel Goal")
goal_sent_button = widgets.Button(description="Display Goal Sent")

def on_cancel_button_clicked(b):
    client.cancel_goal()
    rospy.loginfo("Goal cancelled")
    time.sleep(0.1)
    print("Reinsert the goal and confirm it using the Confirm button")
    x_input.observe(handle_x_input, names='value')
    y_input.observe(handle_y_input, names='value')

    button.on_click(on_button_clicked)

def on_goal_sent_button_clicked(b):
    print("List of goals sent: ", goal_list)

cancel_button.on_click(on_cancel_button_clicked)
goal_sent_button.on_click(on_goal_sent_button_clicked)

left_box = VBox([x_input, y_input, button])
right_box = VBox([cancel_button, goal_sent_button])
HBox([left_box, right_box])



In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')  
        self.ln_target, = plt.plot([], [], 'bo')
        self.x_data, self.y_data = [], []
        self.x_target_data, self.y_target_data = [], []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.ln_target

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def target_callback(self, msg):
        self.y_target_data.append(msg.x)
        self.x_target_data.append(msg.y)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.ln_target.set_data(self.x_target_data, self.y_target_data)
        return self.ln, self.ln_target

In [ ]:
%matplotlib widget

vis = Visualiser()
# subscriber to the topic odom to get the position of the robot 
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)

# subscriber to the topic goal_list to get the target position that is continuously updated
sub_2 = rospy.Subscriber('/goal_list', Pos_vel, vis.target_callback)

ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)
plt.show(block=True)

In [ ]:

global pos_vel
pos_vel = Pos_vel()

labels = ['Reached Goals', 'Not Reached Goals']

fig, ax = plt.subplots()

def position_callback(message):
    global pos_vel
    global first_goal
    global num_reached_goals
    global num_goal_not_reached
    global goal_list
    pos_vel.x = message.pose.pose.position.x
    pos_vel.y = message.pose.pose.position.y
    if first_goal == True:
        if pos_vel.x - 0.5 < x < pos_vel.x + 0.5 and pos_vel.y - 0.5 < y < pos_vel.y + 0.5:
            num_reached_goals = num_reached_goals + 1
            num_goal_not_reached = len(goal_list) - num_reached_goals
            first_goal = False
            
            print("Number of goals reached: ", num_reached_goals)
            print("Number of goals not reached: ", num_goal_not_reached)

            ax.clear()
            values = [num_reached_goals, num_goal_not_reached]
            ax.bar(labels, values)
            ax.set_title('Goals Status')
            ax.set_ylabel('Number of Goals')

sub_3 = rospy.Subscriber('/odom', Odometry, position_callback)

ani_2 = FuncAnimation(fig, position_callback, frames=range(100), repeat=False)
plt.show()

In [ ]:
## PLOT ONLY THE FIRST GOAL NEED TO MODIFY IT

# labels = ['Reached Goals', 'Not Reached Goals']

# fig, ax = plt.subplots()

# def update(num):
#     global num_reached_goals, num_goal_not_reached
#     ax.clear()
#     values = [num_reached_goals, num_goal_not_reached]
#     ax.bar(labels, values)
#     ax.set_title('Goals Status')
#     ax.set_ylabel('Number of Goals')


# ani_2 = FuncAnimation(fig, position_callback, frames=range(100), repeat=False)
# plt.show()


In [ ]:
# ## PLOT ONLY THE FIRST GOAL NEED TO MODIFY IT

# labels = ['Reached Goals', 'Not Reached Goals']

# fig, ax = plt.subplots()

# def update(num):
#     global num_reached_goals, num_goal_not_reached
#     ax.clear()
#     values = [num_reached_goals, num_goal_not_reached]
#     ax.bar(labels, values)
#     ax.set_title('Goals Status')
#     ax.set_ylabel('Number of Goals')


# ani_2 = FuncAnimation(fig, update, frames=range(100), repeat=False)
# plt.show()

In [ ]:
# global pos_vel
# global first_goal
# global num_reached_goals
# global num_goal_not_reached
# if first_goal == True:
#     if pos_vel.x - 0.5 < x < pos_vel.x + 0.5 and pos_vel.y - 0.5 < y < pos_vel.y + 0.5:
#         num_reached_goals = num_reached_goals + 1
#         num_goal_not_reached = len(goal_list) - num_reached_goals
        
#         print("Number of goals reached: ", num_reached_goals)
#         print("Number of goals not reached: ", num_goal_not_reached)
   

TUTTA LA ROBA QUA SOTTO NON VA!!!!

In [ ]:
# class Goal_visualiser:
# 	def __init__(self):
# 		self.fig, self.ax = plt.subplots()
# 		self.ln, = plt.plot([], [], 'ro')
# 		self.x_data, self.y_data = [] , []

# 	def plot_init(self):
# 		self.ax.set_xlim(-10, 10)
# 		self.ax.set_ylim(-10, 10)
# 		return self.ln,

# 	def goal_callback(self, msg):
# 		self.y_data.append(msg.x)
# 		self.x_data.append(msg.y)

# 	def update_plot(self, frame):
# 		self.ln.set_data(self.x_data, self.y_data)
# 		return self.ln,

In [ ]:
# %matplotlib widget

# vis = Goal_visualiser()
# sub = rospy.Subscriber('goal_list', Pos_vel, vis.goal_callback)
# ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
# plt.show(block=True)

In [ ]:
# def cb(msg):
#     print("x: ", msg.x)
#     print("y: ", msg.y)

In [ ]:
# sub = rospy.Subscriber('goal_list', Pos_vel, cb)

In [ ]:
# def action_client():
#     global x
#     global y
#     # client = actionlib.SimpleActionClient('/reaching_goal', second_assignment.msg.PlanningAction)
#     client.wait_for_server()
#     var = True
#     goal_reached = False
#     while not rospy.is_shutdown():
#         rospy.Subscriber("/odom", Odometry, pub_pos_vel)
#         if var == True:

#             # x = float(input("Enter desired x coordinate: "))
#             # y = float(input("Enter desired y coordinate: "))

#             rospy.set_param('/des_pos_x', x)
#             rospy.set_param('/des_pos_y', y)

#             goal = second_assignment.msg.PlanningGoal()
#             goal.target_pose.pose.position.x = x
#             goal.target_pose.pose.position.y = y
#             client.send_goal(goal, feedback_cb=feedback_cb)
#             time.sleep(1) 
            # cancel_goal = input("\nPress c if you want to cancel the goal and any other key to mantain it: ")
            # if cancel_goal == 'c':
            #     client.cancel_goal()
            #     rospy.loginfo("Goal cancelled")
            #     time.sleep(0.1) 
            # else:
            #     var = False
#         time.sleep(2)
#         if goal_reached == False:
#             rospy.loginfo(latest_feedback)
#             if latest_feedback.actual_pose.position.x - 0.5 < x < latest_feedback.actual_pose.position.x + 0.5 and latest_feedback.actual_pose.position.y - 0.5 < y < latest_feedback.actual_pose.position.y + 0.5:
#                 rospy.loginfo("Goal reached")
#                 goal_reached = True
#                 restart = input("\nPress r to restart the program or q to quit: ")
#                 if restart == 'r':
#                     var = True
#                     goal_reached = False
#                 else:
#                     rospy.loginfo("Program terminated")
#                     break

In [ ]:
# try:
#     action_client()
# except rospy.ROSInterruptException:
#     print("program interrupted before completion", file=sys.stderr)